<a href="https://colab.research.google.com/github/gankidisaicharan/Crop-Recommendation-System/blob/main/Gradio_Crop_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 32.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7

In [ ]:
import gradio as gr
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import cv2
import numpy as np
import pathlib
from tensorflow.keras.models import Model, load_model
import gradio as gr
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
le = LabelEncoder()
fert_data = pd.read_csv("/content/drive/MyDrive/PROJECTS/Crop Recommendation system using ML/Dataset/Fertilizer Prediction.csv")
file_path = '/content/drive/MyDrive/PROJECTS/Crop Recommendation system using ML/Dataset/PlantVillage'
data_dir = pathlib.Path(file_path)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))


def crop_recommendation(N, P, K, temperature, humidity, ph, rainfall):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    from sklearn.naive_bayes import GaussianNB

    data = pd.read_csv("/content/drive/MyDrive/PROJECTS/Crop Recommendation system using ML/Dataset/Crop_recommendation.csv")
    X = data.drop('label', axis=1)
    y = data['label']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    nb = GaussianNB()
    nb.fit(X_scaled, y)

    input_data = scaler.transform([[N, P, K, temperature, humidity, ph, rainfall]])
    prediction_prob = nb.predict_proba(input_data)[0]

    prob_crop = sorted(zip(prediction_prob*100, nb.classes_), reverse=True)[:3]
    top3_crops = [f"{crop[1]} ({crop[0]:.2f}%)" for crop in prob_crop]

    return "\n".join(top3_crops)





def predict_image(img_array):
    model = load_model('/content/drive/MyDrive/PROJECTS/Crop Recommendation system using ML/Dataset/models_cnn/model-28-0.9455.hdf5')
    img_array = cv2.resize(img_array, (128, 128))
    img_array = np.array(img_array).reshape(1, 128, 128, 3)
    prediction = model.predict([img_array])
    class_idx = np.argmax(prediction)
    class_name = class_names[class_idx]
    return class_name


def fertilizer_recommendation(crop_type, n, p, k):
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    from sklearn.naive_bayes import GaussianNB

    fert_data = pd.read_csv("/content/drive/MyDrive/PROJECTS/Crop Recommendation system using ML/Dataset/Fertilizer Prediction.csv")
    fert_data=fert_data.drop(['Temparature','Humidity ','Moisture','Soil Type'],axis=1)
    fert_data = fert_data.drop_duplicates()
    le = LabelEncoder()
    fert_data['Crop Type']=le.fit_transform(fert_data['Crop Type'])
    X = fert_data.drop('Fertilizer Name',axis=1)
    y = fert_data['Fertilizer Name']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    crop_type = le.transform([crop_type])[0]
    input_data = pd.DataFrame({
        "Crop Type": [crop_type],
        "Nitrogen": [n],
        "Potassium": [k],
        "Phosphorous": [p]
    })
    fertilizer = nb.predict(input_data)[0]
    explanation=''

    if fertilizer == 'Urea':
        explanation = "Urea is a type of nitrogen fertilizer that provides nitrogen in a form that can be easily taken up by plants. Nitrogen is a key nutrient that is required for plant growth and is often deficient in the soil."
    elif fertilizer == 'DAP':
        explanation = "DAP stands for diammonium phosphate, which is a fertilizer that contains both nitrogen and phosphorus. Phosphorus is another important nutrient that is required for plant growth, particularly for the development of roots, flowers, and fruits."
    elif fertilizer == '14-35-14':
        explanation = "14-35-14 is a type of fertilizer that contains 14% nitrogen, 35% phosphorus, and 14% potassium. This balanced combination of nutrients can help to promote healthy plant growth and development."
    elif fertilizer == '28-28':
        explanation = "28-28 is a type of fertilizer that contains 28% nitrogen and 28% phosphorus. This combination of nutrients can be particularly useful for promoting healthy root development."
    elif fertilizer == '17-17-17':
        explanation = "17-17-17 is a type of fertilizer that contains 17% nitrogen, 17% phosphorus, and 17% potassium. This balanced combination of nutrients can be useful for promoting overall plant growth and development."
    elif fertilizer == '20-20':
        explanation = "20-20 is a type of fertilizer that contains 20% nitrogen and 20% phosphorus. This combination of nutrients can be useful for promoting healthy plant growth and development."
    elif fertilizer == '10-26-26':
        explanation = "10-26-26 is a type of fertilizer that contains 10% nitrogen, 26% phosphorus, and 26% potassium. This combination of nutrients can be particularly useful for promoting flower and fruit development."
    else:
        explanation = "Sorry, no recommendation can be made at this time."

    return fertilizer, explanation






with gr.Blocks('Crop Recommendation, Fertilizer Recommendation, Leaf Disease Prediction') as block:
    gr.Markdown('Crop Recommendation System')
    with gr.Tab('Crop Recommendation'):
        crop_inputs = [
            gr.inputs.Slider(0, 200, default=50, label="Nitrogen"),
            gr.inputs.Slider(0, 200, default=50, label="Phosphorus"),
            gr.inputs.Slider(0, 200, default=50, label="Potassium"),
            gr.inputs.Slider(0, 50, default=25, label="Temperature"),
            gr.inputs.Slider(0, 100, default=50, label="Humidity"),
            gr.inputs.Slider(3.5, 9.0, default=6.5, label="pH"),
            gr.inputs.Slider(0, 250, default=125, label="Rainfall")
        ]
        crop_outputs = gr.outputs.Label()
        text_button = gr.Button("predict")
    with gr.Tab('Fertilizer Recommendation'):
        fert_inputs = [
            gr.inputs.Dropdown(choices=list(fert_data["Crop Type"].unique()), label= "Crop Type"),
            gr.inputs.Slider(minimum=0, maximum=150, default=75, label="Nitrogen (N)"),
            gr.inputs.Slider(minimum=0, maximum=150, default=75, label="Phosphorus (P)"),
            gr.inputs.Slider(minimum=0, maximum=150, default=75, label="Potassium (K)")
          ]
        fert_outputs= [gr.outputs.Label(num_top_classes=1),gr.outputs.Textbox()]
        fert_button = gr.Button("Recommend")
    with gr.Tab('Leaf Disease Prediction'):
        with gr.Row():
          image_inputs = gr.inputs.Image(shape=(128, 128))
          image_outputs = gr.outputs.Label(num_top_classes=len(class_names))
        image_button = gr.Button("Predict")
    text_button.click(crop_recommendation, inputs=crop_inputs, outputs=crop_outputs)
    fert_button.click(fertilizer_recommendation, inputs=fert_inputs, outputs=fert_outputs)
    image_button.click(predict_image, inputs=image_inputs, outputs=image_outputs)


block.launch(debug=True,share=True)


/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:638: UserWarning: Cannot load Crop Recommendation, Fertilizer Recommendation, Leaf Disease Prediction. Caught Exception: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'Crop Recommendation, Fertilizer Recommendation, Leaf Disease Prediction'.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, ple

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://db10b2a2db96df6530.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
